In [ ]:
%pip install bigframes

In [ ]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "key.json"

In [ ]:
PROJECT_ID = "mlops-433612"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from vertexai.resources.preview.feature_store import FeatureGroup
from vertexai.resources.preview.feature_store import utils as fs_utils

FEATURE_GROUP_ID = "test_feature_group"  # @param {type:"string"}
DATASET_ID = "featurestore_demo"
TABLE_ID = "cleaned_table"

BQ_TABLE_URI = f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}'

feature_groups = FeatureGroup.list()

# Find the feature group with the desired ID
fg = next((fg for fg in feature_groups if fg.name == FEATURE_GROUP_ID), None)

if fg:
    print(f"Feature group found: {fg}")
else:
    print(f"Feature group with ID {FEATURE_GROUP_ID} not found.")

In [ ]:
from vertexai.resources.preview.feature_store import Feature

feature_names = ["Name","Age","Sex","Fare","Pclass","SibSp","PassengerId","Survived","Parch","Ticket","Cabin","Embarked"]
features = []

for feature in feature_names:
   f: Feature = fg.get_feature(feature)
   features.append(f)

In [ ]:
import bigframes
import pandas as pd
from vertexai.resources.preview.feature_store import offline_store

entity_df = pd.DataFrame(
  data={
    "entity_id": [f"id-{i}" for i in range(0, 182)],
    "timestamp": [pd.Timestamp("2024-09-15 13:24:22.862434 UTC") for i in range(0, 182)],
  },
)

response = offline_store.fetch_historical_feature_values(
  entity_df=entity_df,
  features=features,
)

In [ ]:
response.to_csv('results.csv', index=False)